In [12]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("Is built with CUDA:", tf.test.is_built_with_cuda())
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.10.0
Is built with CUDA: True
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
import tensorflow as tf

# Check if TensorFlow was built with GPU support
print("Is TensorFlow built with CUDA:", tf.test.is_built_with_cuda())

# List available GPUs
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# Check if TensorFlow can use the GPU
print("Is GPU being used:", len(tf.config.list_physical_devices('GPU')) > 0)


Is TensorFlow built with CUDA: True
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU being used: True


In [14]:
import os
import pickle
import numpy as np
from pathlib import Path

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

## EXTRACT IMAGE FEATURES

In [15]:
model = VGG16()
# restructure the model

model = Model(inputs = model.inputs,outputs=model.layers[-2].output)

In [16]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
# Extract features from Image
from PIL import Image
features = {}
directory = r"E:\projects\Image captioning\data\Images" 

list_dir = os.listdir(directory)

for num,img in enumerate(list_dir):
    img_path = os.path.join(directory, img)
    image = load_img(img_path,target_size=(224,224))
    image = img_to_array(image)
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image)
    image_id = img.split('.')[0]
    features[image_id] = feature
    print(num)

1/1 [==============================] - 27s 27s/step
0
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 21ms/step
2
1/1 [==============================] - 0s 23ms/step
3
1/1 [==============================] - 0s 11ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 21ms/step
6
1/1 [==============================] - 0s 18ms/step
7
1/1 [==============================] - 0s 17ms/step
8
1/1 [==============================] - 0s 14ms/step
9
1/1 [==============================] - 0s 21ms/step
10
1/1 [==============================] - 0s 16ms/step
11
1/1 [==============================] - 0s 15ms/step
12
1/1 [==============================] - 0s 20ms/step
13
1/1 [==============================] - 0s 21ms/step
14
1/1 [==============================] - 0s 18ms/step
15
1/1 [==============================] - 0s 16ms/step
16
1/1 [==============================] - 0s 23ms/step
17
1/1 [===============

In [22]:
pickle.dump(features,open(os.path.join(os.getcwd(),'features.pkl'),'wb'))

# Loading features and captions of images

In [28]:
features_path = r'E:\projects\Image captioning\experiments\features.pkl'
captions_path = r"E:\projects\Image captioning\data\captions.txt"

In [34]:
with open(features_path, 'rb') as file:
    features = pickle.load(file)
with open(captions_path, "rb") as file:
    captions = file.read()
    captions = captions.decode("utf-8") 

In [45]:
mapping = {}
for line in captions.split("\n"):
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]

    image_id = image_id.split('.')[0]

    caption = " ".join(caption)

    if image_id not in mapping:
        mapping[image_id]  = []
    mapping[image_id].append(caption)


In [46]:
len(mapping)

8092

In [47]:
mapping

{'image': ['caption'],
 '1000268201_693b08cb0e': ['A child in a pink dress is climbing up a set of stairs in an entry way .',
  'A girl going into a wooden building .',
  'A little girl climbing into a wooden playhouse .',
  'A little girl climbing the stairs to her playhouse .',
  'A little girl in a pink dress going into a wooden cabin .'],
 '1001773457_577c3a7d70': ['A black dog and a spotted dog are fighting',
  'A black dog and a tri-colored dog playing with each other on the road .',
  'A black dog and a white dog with brown spots are staring at each other in the street .',
  'Two dogs of different breeds looking at each other on the road .',
  'Two dogs on pavement moving toward each other .'],
 '1002674143_1b742ab4b8': ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
  'A little girl is sitting in front of a large painted rainbow .',
  'A small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on 